# Clustering and feature extraction pipeline
### input : all the csv files processed with Preprocessing_voltage.ipynb
### output : extracted features from cluster, ROI wise

### import data, append it to make single dataframe

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.cluster.hierarchy import dendrogram, linkage 
import os
from sklearn.cluster import KMeans

all_files = os.listdir("./data")
csv_files = [file for file in all_files if file.endswith('.csv')]
print(csv_files)

global_min = -0.04
global_max = 0.12

In [ ]:
import pandas as pd
import numpy as np

dataframes_list = []

sample_dict = {}

for file_path in csv_files:
    df = pd.read_csv("./data/"+file_path)
    df = df.drop(df.columns[0], axis=1)
    
    # Extract sample name
    sample_name = '_'.join(file_path.split('_')[:3])
    
    if sample_name not in sample_dict:
        sample_dict[sample_name] = []
    
    sample_dict[sample_name].append(df)

# Normalize data within each sample group
normalized_dataframes_list = []

for sample_name, dfs in sample_dict.items():
    sample_df = pd.concat(dfs, axis=1)
    
    # Perform normalization for the sample
    df_max = min(sample_df.values.max(), 0.1)
    sample_df = sample_df * (0.1 / sample_df.values.max())
    
    # Convert numpy array back to DataFrame to maintain column headers
    sample_df = pd.DataFrame(sample_df, columns=sample_df.columns)
    
    normalized_dataframes_list.append(sample_df)

# Concatenate all normalized dataframes
data = pd.concat(normalized_dataframes_list, axis=1)

# Optionally drop specific columns if needed
segment_error = []
data.drop(columns = segment_error, inplace = True)

### extract features (peak_height, slope(varable name : response_dur for legacy), peak_b(binary) to make a dataframe

#### these features are obtained each stimulation : total of 10

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from scipy.optimize import curve_fit

#prarameter for clustering

param_margin = 2 #how much margin would you allow up to before stim
param_div = 5 #how much frames after stim that you would define as stim response

# thresholds for binary peak sorting
threshold_peak_value = 0.035 
threshold_PSNR = 8


# Assuming 'data' is your DataFrame

stimulus_frames = [50 * x for x in range(1,11)]

# Create new DataFrame to store features

peak_response_df = pd.DataFrame()
idx = 0

for column in data.columns:
    peak_responses = []
    for frame in stimulus_frames:
        window_start = frame - param_margin
        
        window_end = frame + 50 - param_margin
        
        window = data.loc[window_start:window_end, column]
        
        peak_value = window[0 : param_margin + param_div].max()
        
        peak_auc = window[0 : param_margin + param_div].sum()
        
        response_dur = peak_value - part2

        auc = window[0:10].sum()
        
        window_mean = window.mean()
        

        
        MSE_ref = np.mean((part4 - part4.mean())**2)
        PSNR = 20 * np.log10(peak_value / np.sqrt(MSE_ref))
        
        if((peak_value > threshold_peak_value) & (PSNR > threshold_PSNR)):
            peak_b = 1
        else:
            peak_b = 0
 
        peak_responses.extend([peak_value, response_dur, peak_b])


    peak_response_df[column] = peak_responses
    idx += 1

extracted = peak_response_df.T

#### features extracted from each peaks are averaged 

In [ ]:
# get the mean of each feature

extracted['11'] = extracted.iloc[:,np.arange(0,30,3)].mean(axis = 1)
extracted['21'] = extracted.iloc[:,np.arange(1,30,3)].mean(axis = 1)
extracted['32'] = extracted.iloc[:,np.arange(2,30,3)].mean(axis = 1)

### PCA plotting to see overall feature map (actuall clustering is done in HCA)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA


# Standardize the data
scaler = StandardScaler()
scaled_data = scaler.fit_transform(extracted[['11', '21', '32']])


#visualization
pca = PCA(n_components=2)
principal_components = pca.fit_transform(scaled_data)

# Create a DataFrame of the principal components
pca_df = pd.DataFrame(data=principal_components, columns=['PC1', 'PC2'])

# Plotting the PCA results
# Initialize and fit PCA, here we make sure to keep only the first two components for vis
plt.figure(figsize=(8, 6))
plt.scatter(pca_df['PC1'], pca_df['PC2'], alpha=0.5)
plt.xlabel('Principal Component 1')
plt.ylabel('Principal Component 2')
plt.title('PCA Scatter Plot')
plt.grid(True)
plt.show()

# display explained variance
explained_variance = pca.explained_variance_ratio_
print("Explained variance by component:", explained_variance)

### HCA clustering using the scale data from above shell

In [ ]:
import numpy as np
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.cluster import DBSCAN
from sklearn.cluster import AgglomerativeClustering


k = 3

data_t = data.T

# initialization of AggolomertiveClustering and fit
hca = AgglomerativeClustering(n_clusters=k, metric='euclidean', linkage='ward')
labels = hca.fit_predict(scaled_data)                              

# add resulting labels to orginal dataframe
data_t['cluster'] = labels

# output results
print(data_t.head())



### plotting Averaged traces from each cluster (for examination)

In [ ]:
num_points = data_t.shape[1]-1  # Number of time points in your DataFrame
time_vector = np.arange(0, num_points * 0.01, 0.01)  # Create a time vector that matches your data


fig, axs = plt.subplots(k, 1, figsize=(10, k * 3), sharex=True)

global_min = -0.04
global_max = 0.12

for i in range(k):
    cluster_data = data_t[data_t['cluster'] == i].drop('cluster', axis=1)
    print(f'Cluster {i}')
    print(cluster_data.T.columns)
    filename = f'clusteredoutput/original_cluster_{i}.csv'
    cluster_data.T.to_csv(filename)
    
    for index, row in cluster_data.iterrows():
        axs[i].plot(time_vector, row, alpha=0.3, color = 'gray')
    
    axs[i].plot(time_vector, cluster_data.mean(axis=0), alpha = 1, color = 'r')
    
    axs[i].set_title(f'Cluster {i}')
    axs[i].set_xlabel('Time (s)')
    axs[i].set_ylabel('dF/F0')
    axs[i].set_ylim(global_min, global_max)

    

### additional function for mapping specific dataset to integer number

In [ ]:
def datasetmap(date_str):
    
    date_map = {
        '240122_Grid2': 1,
        '240418_Grid1': 2,
        '240529_Grid1': 3,
        '250329_Grid2': 4
    }
    
    return date_map.get(date_str,0)

def dtostr(date, grid, roi, num):
    return str(date) + '_Grid' + str(grid) + '_ROI' + str(roi) + '_#' + str(num)


### PCA plot for examination (including identification whether we got EM)

In [ ]:
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

# include the label if we got EM or not
em_list = pd.read_csv('metadata/metadata.csv')
em_list = em_list['Voltage'].tolist()

count = 0

data_t['em']=0
for row in data_t.iterrows():
    if(row[0] in em_list):
        data_t.loc[row[0],'em']= 1
        count += 1
        
data_t['sample_name'] = data_t.index.str.split('_').str[0] + '_' + data_t.index.str.split('_').str[1]
data_t['sample_name'] = data_t['sample_name'].apply(datasetmap)

data_t.to_csv("metadata/clusterandem.csv")
        
        
# PCA plotting
X = scaled_data
y = data_t['cluster'].values  # Labels
z = data_t['em'].values

# Initialize and fit PCA
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X)

# Plot the results
plt.figure(figsize=(10, 6))
colors = ['r', 'g', 'b','k', 'c', 'y']
for i, color in zip(range(k), colors):
    plt.scatter(X_pca[y == i, 0], X_pca[y == i, 1], c=color, label=f'Cluster {i}')

    
alphas = [0,0.5,0]
for i, color, alpha in zip(range(k), colors, alphas):
    plt.scatter(X_pca[z == i, 0], X_pca[z == i, 1], c='w', alpha = alphas[i], edgecolor='gold', linewidth=1.5, s=100)

for i, (x, y, name) in enumerate(zip(X_pca[:, 0], X_pca[:, 1], sample_name)):
    if data_t.iloc[i]['em'] == 1:
        plt.scatter(x, y, c='w', alpha = 0.5, edgecolor='gold', linewidth=1.5, s=100)
        plt.text(x, y, name, fontsize=9, ha='right')  

plt.title('PCA visualization of the time series clusters')
plt.xlabel('Principal Component 1')
plt.ylabel('Principal Component 2')
plt.legend()
plt.savefig("fig.eps")
plt.show()




### Generate clustering heatmap

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.cluster.hierarchy import dendrogram, linkage
from sklearn.preprocessing import StandardScaler

from matplotlib.colors import LinearSegmentedColormap

colors = [
    (0.0, '#000000'),   # Black at 0.0
    (0.1, '#000000'),  # Black at 0.04
    (0.3, '#8800FF'), # Dark purple at 0.045
    (0.5, '#FF0000'),  # Red at 0.05
    (0.7, '#FFFF00'), # Yellow at 0.055
    (1, "#FFFF00")
]


# Create the colormap
cmap = LinearSegmentedColormap.from_list('my_cmap', colors)

save = False


# Generate the linkage matrix using the Ward method
Z = linkage(scaled_data, method='ward')

# Plotting
fig = plt.figure(figsize=(15, 50))

# Add dendrogram axis
ax_dendro = fig.add_axes([0.09, 0.1, 0.2, 0.6])  # x, y, width, height
dn = dendrogram(Z, orientation='left', ax=ax_dendro, no_plot = True)
ax_dendro.set_xticks([])
ax_dendro.set_yticks([])

dn['leaves'] = dn['leaves'][::-1]
dendrogram(Z, orientation='left', ax=ax_dendro, no_labels=True, leaf_rotation=0, leaf_font_size=12)

# Add heatmap axis
kwargs = {'rasterized' : True}
ax_heatmap = fig.add_axes([0.3, 0.1, 0.6, 0.6])
data_reordered = ((data_t.drop('cluster', axis =1)).drop('em', axis = 1)).drop('sample_name', axis = 1).iloc[dn['leaves']]  # Reorder data based on the dendrogram
sns.heatmap(data_reordered, cmap=cmap, ax=ax_heatmap, vmin=0.00, vmax=0.1, **kwargs, 
            cbar_kws={'label': 'Scale', 'shrink' : 0.2, 'location':'right'})

ax_cluster = fig.add_axes([0.9, 0.1, 0.1,0.6])
data_reordered = data_t.iloc[dn['leaves']]
sns.heatmap(data_reordered[['cluster','em','sample_name']], cmap='rainbow', ax=ax_cluster, vmin=0.0, vmax=6,  **kwargs, 
            cbar_kws={'label': 'cluster, em', 'shrink' : 0.5, 'location' : 'right'})


# Adjusting x-axis to show time in seconds
frame_to_time = lambda x: x * 10 / 1000  # Convert frames to seconds (10 ms per frame)
num_frames = data_reordered.shape[1]
time_ticks = [frame_to_time(x) for x in range(0, num_frames, int(num_frames / 10))]  # Generate time ticks
ax_heatmap.set_xticks(np.linspace(0, num_frames, len(time_ticks)))
ax_heatmap.set_xticklabels(['{:.2f} s'.format(t) for t in time_ticks])
ax_heatmap.set_xlabel('Time (s)')
ax_heatmap.set_ylabel('Feature Index')
ax_heatmap.set_yticks(data_t['cluster']);
ax_cluster.set_yticks([])

plt.savefig("fig.eps", format = 'eps', dpi = 150)

In [ ]:
X = scaled_data
y = data_t['sample_name'].values  # Labels
z = data_t['em'].values

# Initialize and fit PCA
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X)

# Plot the results
plt.figure(figsize=(10, 6))
colors = ['k', 'k', 'k','k', 'k', 'k','k','k','r']
#colors = ['r', 'g', 'b','k', 'c']
for i, color in zip(range(9), colors):
    plt.scatter(X_pca[y == i, 0], X_pca[y == i, 1], c=color, label=f'dataset {i}')
    
    

plt.title('PCA visualization of the time series clusters')
plt.xlabel('Principal Component 1')
plt.ylabel('Principal Component 2')
plt.legend()
plt.savefig("asd.eps")
plt.show()


### Finally, save statistics of each ROIs this includes EM, cluster and integer sample name

In [ ]:
statistics = extracted

statistics['clusters'] = labels
statistics['em'] = data_t['em']
statistics['sample_name'] = data_t['sample_name']

statistics.to_csv("metadata/stat_250628.csv")
